# Train, tune, and deploy a custom ML model using the Time Series Classification (LSTM-FCN) Algorithm from AWS Marketplace

The [Time Series Classification (LSTM-FCN) Algorithm from AWS Marketplace](https://aws.amazon.com/marketplace/pp/prodview-vzxmyw25oqtx6) performs time series classification with the Long Short-Term Memory Fully Convolutional Network (LSTM-FCN) model. It implements both training and inference from CSV data and supports both CPU and GPU instances. The training and inference Docker images were built by extending the PyTorch 2.0 Python 3.10 SageMaker containers. 

#### Model Overview  
The LSTM-FCN model includes two blocks: a recurrent block and a convolutional block.
The recurrent block consists of a single LSTM layer followed by a dropout layer.
The convolutional block consists of three convolutional layers, each followed by batch normalization and ReLU activation, and of a global average pooling layer.
The input time series are passed to both blocks. 
The convolutional block processes each time series as a single feature observed across multiple time steps, while the recurrent block processes each time series as multiple features observed at a single time step (referred to as dimension shuffling).
The outputs of the two blocks are concatenated and passed to a final output layer with softmax activation. 

<img src=https://fg-research-assets.s3.eu-west-1.amazonaws.com/lstm-fcn-diagram.png style="width:80%;margin-top:30px;margin-bottom:20px"/>

*LSTM-FCN architecture (source: [doi: 10.1109/ACCESS.2017.2779939](https://doi.org/10.1109/ACCESS.2017.2779939))*

#### Model Resources 
- **Paper:** [LSTM Fully Convolutional Networks for Time Series Classification](https://doi.org/10.1109/ACCESS.2017.2779939)
- **Repository:** [LSTM-FCN Official GitHub Repository](https://github.com/titu1994/LSTM-FCN)

This sample notebook shows you how to train a custom ML model using the [Time Series Classification (LSTM-FCN) Algorithm from AWS Marketplace](https://aws.amazon.com/marketplace/pp/prodview-vzxmyw25oqtx6).

**Note: This is a reference notebook and it cannot run unless you make the changes suggested in the notebook.**

## Pre-requisites
1. This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to the [Time Series Classification (LSTM-FCN) Algorithm from AWS Marketplace](https://aws.amazon.com/marketplace/pp/prodview-vzxmyw25oqtx6).

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize train and test dataset](#B.-Configure-and-visualize-train-and-test-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train a machine learning model](#3:-Train-a-machine-learning-model)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-a-model)
1. [Deploy model and verify results](#4:-Deploy-model-and-verify-results)
    1. [Deploy trained model](#A.-Deploy-trained-model)
    1. [Create input payload](#B.-Create-input-payload)
    1. [Perform real-time inference](#C.-Perform-real-time-inference)
    1. [Visualize output](#D.-Visualize-output)
    1. [Calculate relevant metrics](#E.-Calculate-relevant-metrics)
    1. [Delete the endpoint](#F.-Delete-the-endpoint)
1. [Tune your model! (optional)](#5:-Tune-your-model!-(optional))
	1. [Tuning Guidelines](#A.-Tuning-Guidelines)
	1. [Define Tuning configuration](#B.-Define-Tuning-configuration)
	1. [Run a model tuning job](#C.-Run-a-model-tuning-job)
1. [Perform Batch inference](#6.-Perform-Batch-inference)
1. [Clean-up](#7.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page.
1. On the AWS Marketplace listing, click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on the **Continue to configuration** button and then choose a **region**, you will see a **Product ARN**. This is the algorithm ARN that you need to specify while training a custom ML model. **Copy the ARN corresponding to your region and specify the same in the following cell.**

In [1]:
algo_arn = "arn:aws:sagemaker:eu-west-1:661670223746:algorithm/lstm-fcn-algo"

## 2. Prepare dataset

In [2]:
import io
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

In [3]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
bucket

'sagemaker-eu-west-1-661670223746'

### A. Dataset format expected by the algorithm

The training and validation datasets should be provided as CSV files. 
The datasets should contain the time series and their class labels.
Each row of the CSV file represents a time series, while each column represents a time step.
All time series should have the same length and should not contain missing values.

The class labels should be stored in the first column, while the time series should be stored in the subsequent columns.
The class labels should be integers from `0` to `N - 1`, where `N` is the number of classes.
The CSV file should not contain any index column or column headers. 

You can find more information about dataset format in the **Usage Information** section of the algorithm listing page.

### B. Configure and visualize train and test dataset

The sample data provided in the algorithm's GitHub repository (https://github.com/fg-research/lstm-fcn-sagemaker) consists of three classes of artificially generated time series. The length of each time series is 100 time steps. The training dataset contains 700 time series, while the validation and test datasets contain 150 time series. 

Training dataset:

In [4]:
training_dataset = pd.read_csv(
    "https://raw.githubusercontent.com/fg-research/lstm-fcn-sagemaker/master/data/training/train.csv",
    header=None,
    index_col=None
)

In [5]:
training_dataset.shape

(700, 101)

In [6]:
training_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,-9.653245,12.503635,0.364683,8.849870,15.112630,13.802204,10.928423,3.953502,20.437704,...,-1.296008,-0.142416,26.657452,26.845801,21.094451,28.482447,-3.000038,-5.253792,6.692845,7.355366
1,2.0,1.635787,-0.679125,-10.791860,13.123179,25.574548,32.136099,20.859789,15.460488,20.785099,...,-2.857502,-7.066338,22.722743,29.849994,30.200593,-5.622287,8.211991,13.657285,7.778116,10.592168
2,1.0,7.058552,-2.623195,26.745231,22.265812,3.441419,3.934305,-3.726288,-2.348845,17.311256,...,-3.714819,12.211388,40.242840,20.876647,17.800351,5.946556,-22.282134,20.586275,23.964243,-2.485860
3,1.0,-10.069621,-8.714222,16.213594,3.487754,-8.008801,27.680261,3.248010,-1.967342,8.183699,...,-0.197708,-5.610286,14.377007,21.418152,-7.947945,3.909288,-0.677026,-7.557458,14.940218,13.796720
4,1.0,18.301973,14.657118,-11.845406,10.525305,17.020131,-0.417659,-2.370331,-10.648123,25.889483,...,1.196496,5.505215,7.663839,5.822160,24.019266,10.538759,7.596436,24.862538,38.692663,4.645963


In [7]:
training_dataset.iloc[:, 0].sort_values().unique()

array([0., 1., 2.])

Validation dataset:

In [8]:
validation_dataset = pd.read_csv(
    "https://raw.githubusercontent.com/fg-research/lstm-fcn-sagemaker/master/data/training/valid.csv",
    header=None,
    index_col=None
)

In [9]:
validation_dataset.shape

(150, 101)

In [10]:
validation_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1.0,15.478889,6.773570,-10.307197,24.924477,19.201765,16.836480,-8.235487,-9.037011,13.581153,...,-18.169935,12.032606,17.552417,14.037278,6.454744,-14.332159,13.894731,7.677091,24.552010,0.380896
1,1.0,22.591914,18.728685,3.226004,10.545205,41.212151,20.576582,0.406792,11.362842,6.217928,...,0.064311,3.098154,23.039155,17.979268,9.616904,-13.297607,-8.835471,18.541289,25.855706,7.395281
2,0.0,2.651071,-2.657937,3.648929,7.510428,37.409361,34.633973,23.413473,25.917828,14.415751,...,-12.787993,1.625704,18.428298,-5.561938,35.923486,19.942426,26.873630,-1.828905,6.985897,0.851959
3,2.0,-1.838490,-17.546395,15.898714,22.614802,23.301109,23.542851,16.769920,15.424828,-9.275713,...,-13.446913,-7.111415,21.063856,30.736190,20.752074,18.309628,4.356676,17.463315,-1.107846,19.377706
4,1.0,18.064992,11.531259,6.296674,-4.855813,9.530433,13.590807,-6.319849,-8.786876,14.091134,...,-8.266969,9.832693,4.932949,32.174798,8.449398,-5.938063,9.212065,16.015061,27.147949,-10.010569


In [11]:
validation_dataset.iloc[:, 0].sort_values().unique()

array([0., 1., 2.])

### C. Upload datasets to Amazon S3

In [12]:
training_data = sagemaker_session.upload_string_as_file_body(
    body=training_dataset.to_csv(index=False, header=False),
    bucket=bucket, 
    key="lstm-fcn-algo/data/training/input/train.csv"
)

In [13]:
training_data

's3://sagemaker-eu-west-1-661670223746/lstm-fcn-algo/data/training/input/train.csv'

In [14]:
validation_data = sagemaker_session.upload_string_as_file_body(
    body=validation_dataset.to_csv(index=False, header=False),
    bucket=bucket, 
    key="lstm-fcn-algo/data/training/input/valid.csv"
)

In [15]:
validation_data

's3://sagemaker-eu-west-1-661670223746/lstm-fcn-algo/data/training/input/valid.csv'

## 3: Train a machine learning model

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [16]:
role = sagemaker.get_execution_role()

### 3.2 Train a model

Given that the sample dataset is relatively small, we keep the number of parameters low and we run the training for only a few epochs. 

You can find more information about the model's hyperparameters in the **Hyperparameters** section of the algorithm listing page.

In [17]:
# define hyperparameters
hyperparameters = {
    "units": 8,
    "dropout": 0.8,
    "filters-1": 4,
    "filters-2": 4,
    "filters-3": 4,
    "kernel-size-1": 3,
    "kernel-size-2": 3,
    "kernel-size-3": 3,
    "batch-size": 64,
    "lr": 0.001,
    "epochs": 10,
}

For information on creating an `Estimator` object, see the [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html).

In [18]:
# create an estimator object for running a training job
estimator = sagemaker.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="lstm-fcn-training",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    input_mode="File",
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
)

# run the training job
estimator.fit({"training": training_data, "validation": validation_data})

INFO:sagemaker:Creating training-job with name: lstm-fcn-training-2023-07-21-18-50-04-621


2023-07-21 18:50:04 Starting - Starting the training job...
2023-07-21 18:50:19 Starting - Preparing the instances for training......
2023-07-21 18:51:31 Downloading - Downloading input data
2023-07-21 18:51:31 Training - Downloading the training image..............................
2023-07-21 18:56:27 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-07-21 18:56:33,238 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-07-21 18:56:33,239 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-07-21 18:56:33,240 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-07-21 18:56:33,249 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-07-21 18:56:33,255 sagemaker_pytorch_container.training INFO    

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model and verify results

Now you can deploy the model for performing real-time inference.

### A. Deploy trained model

In [19]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    serializer=sagemaker.serializers.CSVSerializer(content_type="text/csv")
)

INFO:sagemaker:Creating model package with name: lstm-fcn-training-2023-07-21-18-57-19-009


.........

INFO:sagemaker:Creating model with name: lstm-fcn-training-2023-07-21-18-57-19-009


INFO:sagemaker:Creating endpoint-config with name lstm-fcn-training-2023-07-21-18-57-19-009
INFO:sagemaker:Creating endpoint with name lstm-fcn-training-2023-07-21-18-57-19-009


---------!

Once the endpoint is in service, you can perform real-time inference.

### B. Create input payload

The inference algorithm takes as input a CSV file containing the time series.
Each row of the CSV file represents a time series, while each column represents a time step.
The CSV file should not contain any index column or column headers.
All time series should have the same length and should not contain missing values.

Test data:

In [20]:
test_dataset = pd.read_csv(
    "https://raw.githubusercontent.com/fg-research/lstm-fcn-sagemaker/master/data/inference/input/test_data.csv",
    header=None,
    index_col=None
)

In [21]:
test_dataset.shape

(150, 100)

In [22]:
test_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,13.922934,-12.333286,16.970006,20.459903,10.755518,5.953969,15.223849,9.021735,7.018893,11.261269,...,-6.807806,-11.919346,11.581619,32.449001,2.507668,10.808019,1.193405,33.232719,42.003243,20.314625
1,2.305337,-5.799543,28.457907,27.666659,10.818559,18.721389,20.306512,21.298879,6.722657,-8.661484,...,9.424779,-7.549842,10.220740,23.419116,17.119124,16.169222,11.892821,-1.325216,13.686307,-10.180377
2,13.141899,-11.770865,-0.760004,42.355950,17.172389,26.767570,-7.531622,-1.133515,20.472634,-3.463446,...,-27.063771,0.814035,17.655639,22.273710,0.180531,13.680891,16.564715,3.861534,2.558995,-3.174557
3,-18.277705,-19.144148,1.763139,21.252543,26.622625,27.344010,7.139832,17.754067,16.333792,-0.494965,...,-11.311305,14.902996,26.646289,19.877144,31.825011,11.897166,1.253178,-4.224470,19.184084,-7.240560
4,-2.431898,-11.903490,-3.126534,25.505996,36.183348,15.120651,3.765192,1.706296,23.100381,19.476364,...,-20.407485,-3.496949,-7.278537,31.580566,30.925944,25.387130,4.938499,15.018984,11.352280,18.085493


In [23]:
test_data = sagemaker_session.upload_string_as_file_body(
    body=test_dataset.to_csv(index=False, header=False),
    bucket=bucket, 
    key="lstm-fcn-algo/data/inference/input/test_data.csv"
)

In [24]:
payload = sagemaker_session.read_s3_file(
    bucket=bucket, 
    key_prefix="lstm-fcn-algo/data/inference/input/test_data.csv"
)

### C. Perform real-time inference

In [25]:
response = sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType='text/csv',
    Body=payload
)
print(response)
real_time_predictions = response['Body'].read().decode('utf-8').split('\n')[:-1]
real_time_predictions = pd.DataFrame(real_time_predictions).astype(float)

{'ResponseMetadata': {'RequestId': '578a825a-2345-449b-9efa-2770a91ef093', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '578a825a-2345-449b-9efa-2770a91ef093', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Fri, 21 Jul 2023 19:03:07 GMT', 'content-type': 'application/json', 'content-length': '300', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f27a2d6eda0>}


In [26]:
real_time_results = sagemaker_session.upload_string_as_file_body(
    body=real_time_predictions.to_csv(index=False, header=False),
    bucket=bucket, 
    key="lstm-fcn-algo/data/inference/output/real-time/real_time_predictions.csv"
)

In [27]:
real_time_results

's3://sagemaker-eu-west-1-661670223746/lstm-fcn-algo/data/inference/output/real-time/real_time_predictions.csv'

### D. Visualize output

The inference algorithm outputs the predicted class labels in the same format as they were provided for training, that is as integers from `0` to `N - 1`, where `N` is the number of classes. The predicted class labels are returned in CSV format.

In [28]:
real_time_predictions.shape

(150, 1)

In [29]:
real_time_predictions.head()

,0
0,1.0
1,0.0
2,2.0
3,2.0
4,2.0


In [30]:
print(real_time_predictions.iloc[:, 0].sort_values().unique())

[0. 1. 2.]


### E. Calculate relevant metrics

Ground truth test labels:

In [31]:
test_labels = pd.read_csv(
    "https://raw.githubusercontent.com/fg-research/lstm-fcn-sagemaker/master/data/inference/output/ground-truth/test_labels.csv",
    header=None,
    index_col=None
)

In [32]:
test_labels.shape

(150, 1)

In [33]:
test_labels.head()

,0
0,1.0
1,0.0
2,2.0
3,2.0
4,2.0


In [34]:
print(test_labels.iloc[:, 0].sort_values().unique())

[0. 1. 2.]


In [35]:
accuracy_score(y_true=test_labels.iloc[:, 0], y_pred=real_time_predictions.iloc[:, 0])

0.9333333333333333

In [36]:
pd.DataFrame(classification_report(y_true=test_labels.iloc[:, 0], y_pred=real_time_predictions.iloc[:, 0], output_dict=True))

,0.0,1.0,2.0,accuracy,macro avg,weighted avg
precision,0.938776,1.000000,0.870370,0.933333,0.936382,0.936382
recall,0.920000,0.940000,0.940000,0.933333,0.933333,0.933333
f1-score,0.929293,0.969072,0.903846,0.933333,0.934070,0.934070
support,50.000000,50.000000,50.000000,0.933333,150.000000,150.000000


If [Amazon SageMaker Model Monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor.html) supports the type of problem you are trying to solve using this algorithm, use the following examples to add Model Monitor support to your product.
For sample code to enable and monitor the model, see following notebooks:
1. [Enable Amazon SageMaker Model Monitor](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_model_monitor/enable_model_monitor/SageMaker-Enable-Model-Monitor.ipynb)
2. [Amazon SageMaker Model Monitor - visualizing monitoring results](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker_model_monitor/visualization/SageMaker-Model-Monitor-Visualize.ipynb)

### F. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the same to avoid being charged.

In [37]:
predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: lstm-fcn-training-2023-07-21-18-57-19-009
INFO:sagemaker:Deleting endpoint with name: lstm-fcn-training-2023-07-21-18-57-19-009


Since this is an experiment, you do not need to run a hyperparameter tuning job. However, if you would like to see how to tune a model trained using a third-party algorithm with Amazon SageMaker's hyperparameter tuning functionality, you can run the optional tuning step.

## 5: Tune your model! (optional)

### A. Tuning Guidelines

The model has the following hyperparameters, all of which are tunable:
- `units`: `int`. The number of units of the LSTM layer (default = 8).
- `dropout`: `float`. The rate of the dropout layer (default = 0.8).
- `filters-1`: `int`. The number of filters of the first convolutional layer (default = 128).
- `filters-2`: `int`. The number of filters of the second convolutional layer (default = 256).
- `filters-3`: `int`. The number of filters of the third convolutional layer (default = 128).
- `kernel-size-1`: `int`. The size of the kernel of the first convolutional layer (default = 8).
- `kernel-size-2`: `int`. The size of the kernel of the second convolutional layer (default = 5).
- `kernel-size-3`: `int`. The size of the kernel of the third convolutional layer (default = 3).
- `lr`: `float`. The learning rate used for training (default = 0.001).
- `batch-size`: `int`. The batch size used for training (default = 128).
- `epochs`: `int`. The number of training epochs (default = 2000).

### B. Define Tuning configuration

Given that the sample dataset is relatively small, we keep the number of parameters low and we run the training for only a few epochs. 

In [38]:
hyperparameter_ranges = {
    "units": sagemaker.parameter.IntegerParameter(4, 16),
    "dropout": sagemaker.parameter.ContinuousParameter(0.1, 0.9),
    "filters-1": sagemaker.parameter.IntegerParameter(4, 16),
    "filters-2": sagemaker.parameter.IntegerParameter(4, 16),
    "filters-3": sagemaker.parameter.IntegerParameter(4, 16),
    "kernel-size-1": sagemaker.parameter.IntegerParameter(3, 7),
    "kernel-size-2": sagemaker.parameter.IntegerParameter(3, 7),
    "kernel-size-3": sagemaker.parameter.IntegerParameter(3, 7),
    "lr": sagemaker.parameter.ContinuousParameter(0.0001, 0.001),
    "batch-size": sagemaker.parameter.CategoricalParameter([32, 64, 128]),
    "epochs": sagemaker.parameter.IntegerParameter(5, 10),
}

We use the validation accuracy as the objective to be maximized.

In [39]:
objective_metric_name = "valid:acc"

In [40]:
objective_type = "Maximize"

### C. Run a model tuning job

In the interest of time, we run the tuner only for a few iterations. 

In [41]:
tuner = sagemaker.tuner.HyperparameterTuner(
    estimator=estimator,
    base_tuning_job_name="lstm-fcn-tuning",
    objective_metric_name=objective_metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    max_jobs=4,
    max_parallel_jobs=4,
)

In [42]:
tuner.fit({"training": training_data, "validation": validation_data})

INFO:sagemaker:Creating hyperparameter tuning job with name: lstm-fcn-tuning-230721-1903


........................................................................................!


In [43]:
tuner.analytics().dataframe()

,batch-size,dropout,epochs,filters-1,filters-2,filters-3,kernel-size-1,kernel-size-2,kernel-size-3,lr,units,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,128.0,0.445160,6.0,13.0,6.0,5.0,4.0,4.0,6.0,0.000957,14.0,lstm-fcn-tuning-230721-1903-004-7b20c3ab,Completed,0.8333,2023-07-21 19:04:26+00:00,2023-07-21 19:09:49+00:00,323.0
1,128.0,0.285118,10.0,4.0,4.0,4.0,7.0,4.0,6.0,0.000910,6.0,lstm-fcn-tuning-230721-1903-003-099f4dcb,Completed,0.8267,2023-07-21 19:04:18+00:00,2023-07-21 19:09:45+00:00,327.0
2,32.0,0.812971,5.0,11.0,14.0,12.0,6.0,5.0,3.0,0.000676,5.0,lstm-fcn-tuning-230721-1903-002-935b558b,Completed,0.7333,2023-07-21 19:04:14+00:00,2023-07-21 19:09:52+00:00,338.0
3,32.0,0.633893,8.0,16.0,13.0,13.0,7.0,5.0,4.0,0.000982,5.0,lstm-fcn-tuning-230721-1903-001-b8ee5a2d,Completed,0.8800,2023-07-21 19:04:12+00:00,2023-07-21 19:09:50+00:00,338.0


Once you have completed a tuning job (or even while the job is still running), you can [clone and use this notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) to analyze the results to understand how each hyperparameter effects the quality of the model.

## 6. Perform Batch inference

In this section, you will perform batch inference.

In [44]:
transformer = estimator.transformer(
    instance_count=1, 
    instance_type="ml.m5.xlarge",
)

INFO:sagemaker:Creating model package with name: lstm-fcn-algo-2023-07-21-19-10-43-645


.........

INFO:sagemaker:Creating model with name: lstm-fcn-algo-2023-07-21-19-10-43-645-2023-07-21-19-11-29-048


In [45]:
transformer.transform(
    data=test_data,
    content_type='text/csv'
)

INFO:sagemaker:Creating transform job with name: lstm-fcn-training-2023-07-21-19-11-29-799


2023-07-21T19:19:03,706 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2023-07-21T19:19:03,767 [INFO ] main org.pytorch.serve.metrics.configuration.MetricConfiguration - Successfully loaded metrics configuration from /opt/conda/lib/python3.10/site-packages/ts/configs/metrics.yaml
2023-07-21T19:19:03,876 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.8.0
TS Home: /opt/conda/lib/python3.10/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Metrics config path: /opt/conda/lib/python3.10/site-packages/ts/configs/metrics.yaml
Number of GPUs: 0
Number of CPUs: 4
Max heap size: 3934 M
Python executable: /opt/conda/bin/python3.10
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty thread

In [46]:
transformer.output_path

's3://sagemaker-eu-west-1-661670223746/lstm-fcn-training-2023-07-21-19-11-29-799'

In [47]:
batch_predictions = sagemaker_session.read_s3_file(
    bucket=bucket, 
    key_prefix=f"{transformer.output_path.split('/')[-1]}/test_data.csv.out"
)

batch_predictions = batch_predictions.split('\n')[:-1]
batch_predictions = pd.DataFrame(batch_predictions).astype(float)
batch_predictions.head()

,0
0,1.0
1,0.0
2,2.0
3,2.0
4,2.0


In [48]:
np.all(batch_predictions.values == real_time_predictions.values)

True

In [49]:
batch_results = sagemaker_session.upload_string_as_file_body(
    body=batch_predictions.to_csv(index=False, header=False),
    bucket=bucket, 
    key="lstm-fcn-algo/data/inference/output/batch/batch_predictions.csv"
)

## 7. Clean-up

### A. Delete the model

In [50]:
transformer.delete_model()

INFO:sagemaker:Deleting model with name: lstm-fcn-algo-2023-07-21-19-10-43-645-2023-07-21-19-11-29-048


### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

